In [1]:
# IMPORT ALL RELEVANT LIBRARIES

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
import re

In [2]:
### DRIVER SETUP

# initialise the driver (and open up a browser window)
driver = webdriver.Chrome(r"C:\Users\MattiaDevescovi\Desktop\chromedriver.exe")

# open up a specific web page
driver.get("https://it.everli.com/it/")

C:\Users\MattiaDevescovi\AppData\Local\Temp\ipykernel_2240\2124845867.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\MattiaDevescovi\Desktop\chromedriver.exe")


In [3]:
### SCRAPE NAME, DESCRITION, PRICE AND IMAGE OF ALL THE PRODUCTS
## DEFINE FUNCTION FOR COLUMN LIST CREATION (names, description and price)
def create_col(class_name):
    s_column = driver.find_elements(By.CLASS_NAME, class_name)
    column = [col.text for col in s_column]
    return column

## CREATE A LIST WITH ALL LINKS IN THE CATEGORY PAGE
# Find all the links on the page
links_category = driver.find_elements(By.TAG_NAME, "a")
# Create an empty list to store the URLs
urls_category = []
# Iterate over the links and add the URLs to the list
for link_category in links_category:
    url_category = link_category.get_attribute("href")
    urls_category.append(url_category)
# Filter the list of URLs to only keep the ones that contain the specific string
filtered_urls_category = [url_category for url_category in urls_category if "https://it.everli.com/s#/locations/12307/stores/7142/categories/" in url_category]
# Create a set from filtered_urls list
unique_urls_category = set(filtered_urls_category)
# Convert the set back to list
filtered_urls_category = list(unique_urls_category)
# Print the filtered list of URLs
print(filtered_urls_category)

## START ITERATING ON EVERY PAGE THAT NEEDS TO BE SCRAPED 
# Iterate and click on every link in the list
for index_category, url_category in enumerate(filtered_urls_category):
    urls_category = "'"+url_category+"'"
    driver.get(url_category)
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//h1[@class='category-name']")))
    time.sleep(3)

    ## GET 1st LEVEL OF CATEGORIZATION TO APPEND THE CATEGORY COLUMN IN THE DATAFRAME
    Category_name = driver.find_element(By.CLASS_NAME, 'category-name')
    Category = Category_name.text
    
    ## CREATE A LIST WITH ALL LINKS IN THE SUBCATEGORY PAGE
    # Find all the links on the page
    links = driver.find_elements(By.TAG_NAME, "a")
    # Create an empty list to store the URLs
    urls = []
    # Iterate over the links and add the URLs to the list
    for link in links:
        url = link.get_attribute("href")
        urls.append(url)
    # Filter the list of URLs to only keep the ones that contain the specific string
    filtered_urls = [url for url in urls if "https://it.everli.com/s#/locations/12307/stores/7142/categories/" in url]
    # Create a set from filtered_urls list
    unique_urls = set(filtered_urls)
    # Convert the set back to list
    filtered_urls = list(unique_urls)
    # Mi salvo la lunghezza della lista, mi serve per capire quando sono all'ultima iterazione
    last_index = len(filtered_urls)
    # Print the filtered list of URLs
    print(filtered_urls)
    time.sleep(3)
    
    # Iterate and click on every link in the list 
    for index, url in enumerate(filtered_urls):
        urls = "'"+url+"'"
        driver.get(url)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//h1[@class='category-name']")))
        time.sleep(3)

        ##SCRAPING STARTS
        # Scroll down untill the end of the page to be sure that all the content will be scraped
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

        # create a list for name, description and price columns of the table
        name = create_col('name')
        description = create_col('description')
        price = create_col('price')

        ## IMAGE URL
        # get the html code containing the product ids
        html_tag = driver.find_elements(By.XPATH, '//script[@type="application/ld+json"]')
        html_image = [l.get_attribute('outerHTML') for l in html_tag]
        image_pattern = r'"image": "(https://.+?)"'
        # Create the empty column for product ids
        image = []
        # Use regular expression to extract image url from each string
        for html in html_image:
            match = re.search(image_pattern, html)
            if match:
                image_url = match.group(1);
                image.append(image_url)

        ## PRODUCT ID
        #get the html code containing the product ids
        html_tag = driver.find_elements(By.XPATH, '//script[@type="application/ld+json"]')
        html_products = [l.get_attribute('outerHTML') for l in html_tag]
        # Create the empty column for product ids
        product_ids = []
        # Use regular expression to extract product ID from each string
        for html in html_products:
            match = re.search(r'"productID": "(\d+)"', html)
            if match:
                product_id = match.group(1)
                product_ids.append(product_id)

        ## 2nd LEVEL OF CATEGORIZATION
        Sub_Category = driver.find_element(By.CLASS_NAME, 'category-name')
        Sub_Category_text = Sub_Category.text


        ### STRUCTURE SUBCATEGORY INTO A DATAFRAME
        if index == 0:
            partner_menu_subcat = pd.DataFrame(
                {'name': name,
                 'description': description,
                 'price': price, 
                 'image': image,
                 'external_ID': product_ids,
                 'Sub Category': Sub_Category_text,
                 'Category': Category
                })
        if index != 0:
            temp_partner_menu_subcat = pd.DataFrame(
                {'name': name,
                 'description': description,
                 'price': price,
                 'image': image,
                 'external_ID': product_ids,
                 'Sub Category': Sub_Category_text,
                 'Category': Category
                })
            data = [partner_menu_subcat, temp_partner_menu_subcat]
            partner_menu_subcat = pd.concat(data)
        if index == last_index - 1:
            if index_category == 0:
                partner_menu = partner_menu_subcat
            if index_category != 0:
                data = [partner_menu, partner_menu_subcat]
                partner_menu = pd.concat(data)

        time.sleep(3)

['https://it.everli.com/s#/locations/12307/stores/7142/categories/27', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/101366', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/6', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/12', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/8', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/4', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/20', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/10', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/21', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/1', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/32', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/15', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/11', 'https://it.everli.com/s#/locations/12307/stores/7142/categories/17', 'https://it.everli.

In [4]:
# Reset the Index of the compound Dataframe
partner_menu = partner_menu.reset_index(drop=True)
partner_menu

,name,description,price,image,external_ID,Sub Category,Category
0,D'amico I Facili da Cucinare Funghi Pleurotus,Lattina 400g,"2,23 €",https://d2f5fuie6vdmie.cloudfront.net/asset/im...,131179974,"Olive, Sottoli e Sottaceti","Sughi, Scatolame e Condimenti"
1,D'amico Il Contorno Pronto Friarielli alla Nap...,Vasetto 290g,"4,23 €",https://d2f5fuie6vdmie.cloudfront.net/asset/im...,131180114,"Olive, Sottoli e Sottaceti","Sughi, Scatolame e Condimenti"
2,D'amico Le Specialità Champignon alla Napoletana,Vasetto 280g,"3,70 €",https://d2f5fuie6vdmie.cloudfront.net/asset/im...,131177704,"Olive, Sottoli e Sottaceti","Sughi, Scatolame e Condimenti"
3,D'amico Le Specialità Funghi Misti alla Calabrese,Vasetto 280g,"3,70 €",https://d2f5fuie6vdmie.cloudfront.net/asset/im...,131179674,"Olive, Sottoli e Sottaceti","Sughi, Scatolame e Condimenti"
4,Ponti Ponti Funghi alla Pizzaiola 280 g,barattolo 280 gr,"2,85 €",https://d2f5fuie6vdmie.cloudfront.net/asset/im...,131173268,"Olive, Sottoli e Sottaceti","Sughi, Scatolame e Condimenti"
...,...,...,...,...,...,...,...
1217,Le Naturelle Fresche da Allevamento a Terra Medie,Confezione 6 uova,"2,00 €",https://d2f5fuie6vdmie.cloudfront.net/asset/it...,183162216,Uova,"Uova, Farine e Preparati"
1218,Quaja Veneta 18 Uova di Quaglia Fresche,Confezione 200 g,"4,23 €",https://d2f5fuie6vdmie.cloudfront.net/asset/it...,167353828,Uova,"Uova, Farine e Preparati"
1219,Le Naturelle 6+2 Uova Fresche da Allevamento a...,Confezione 380 g,"2,43 €",https://d2f5fuie6vdmie.cloudfront.net/asset/it...,177687984,Uova,"Uova, Farine e Preparati"
1220,Le Naturelle Albume d'Uovo Pastorizzato,Confezione 500 g,"2,00 €",https://d2f5fuie6vdmie.cloudfront.net/asset/it...,180105858,Uova,"Uova, Farine e Preparati"


In [6]:
# Convert partner_menu DataFrame into a csv
path_csv = (r"C:\Users\MattiaDevescovi\Desktop\Python Scripts\\")
partner_menu.to_csv(path_csv + 'Menu Decò Everli Roma.csv')